# 📈 性能分析与可视化

本笔记本展示如何分析回测结果并生成专业的性能报告。

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ml_framework import FeatureEngineer, MLSignalPredictor, Backtester

## 1. 加载回测结果

In [ ]:
# 加载交易记录
trades_path = Path.cwd().parent / 'results' / 'topix_example_trades.csv'

if trades_path.exists():
    trades = pd.read_csv(trades_path)
    trades['entry_time'] = pd.to_datetime(trades['entry_time'])
    trades['exit_time'] = pd.to_datetime(trades['exit_time'])
    print(f'✅ 加载 {len(trades):,} 笔交易记录')
    print(f'   时间范围: {trades["entry_time"].min()} 到 {trades["exit_time"].max()}')
else:
    print('⚠️ 交易记录文件不存在')
    print('   请先运行: python examples/topix_example.py')

## 2. 基础统计分析

In [ ]:
if trades_path.exists():
    # 计算关键指标
    total_trades = len(trades)
    winning_trades = len(trades[trades['pnl'] > 0])
    losing_trades = len(trades[trades['pnl'] <= 0])
    win_rate = winning_trades / total_trades * 100
    
    avg_win = trades[trades['pnl'] > 0]['pnl'].mean()
    avg_loss = trades[trades['pnl'] <= 0]['pnl'].mean()
    
    total_profit = trades[trades['pnl'] > 0]['pnl'].sum()
    total_loss = abs(trades[trades['pnl'] <= 0]['pnl'].sum())
    profit_factor = total_profit / total_loss if total_loss > 0 else 0
    
    # 显示统计
    print('='*70)
    print('交易统计摘要')
    print('='*70)
    print(f'总交易数:     {total_trades:,}')
    print(f'盈利交易:     {winning_trades:,} ({winning_trades/total_trades*100:.1f}%)')
    print(f'亏损交易:     {losing_trades:,} ({losing_trades/total_trades*100:.1f}%)')
    print(f'胜率:         {win_rate:.2f}%')
    print(f'\n平均盈利:     ¥{avg_win:,.0f}')
    print(f'平均亏损:     ¥{avg_loss:,.0f}')
    print(f'盈亏比:       {abs(avg_win/avg_loss):.2f}' if avg_loss != 0 else 'N/A')
    print(f'\n总盈利:       ¥{total_profit:,.0f}')
    print(f'总亏损:       ¥{total_loss:,.0f}')
    print(f'净利润:       ¥{total_profit-total_loss:,.0f}')
    print(f'盈利因子:     {profit_factor:.2f}')
    print(f'\n平均持仓时间: {trades["bars_held"].mean():.1f} 根K线')
    print('='*70)

## 3. 权益曲线

In [ ]:
if trades_path.exists():
    # 计算累计收益
    initial_capital = 5_000_000
    trades['cumulative_pnl'] = trades['pnl'].cumsum()
    trades['equity'] = initial_capital + trades['cumulative_pnl']
    
    # 绘制权益曲线
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # 权益曲线
    axes[0].plot(trades['exit_time'], trades['equity'], linewidth=2, color='steelblue')
    axes[0].axhline(y=initial_capital, color='red', linestyle='--', alpha=0.5, label='初始资金')
    axes[0].fill_between(trades['exit_time'], initial_capital, trades['equity'], 
                         alpha=0.2, color='green', where=(trades['equity'] >= initial_capital))
    axes[0].fill_between(trades['exit_time'], initial_capital, trades['equity'], 
                         alpha=0.2, color='red', where=(trades['equity'] < initial_capital))
    axes[0].set_title('权益曲线', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('账户权益 (¥)', fontsize=12)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'¥{x/1e6:.1f}M'))
    
    # 回撤曲线
    running_max = trades['equity'].cummax()
    drawdown = trades['equity'] - running_max
    drawdown_pct = (drawdown / running_max) * 100
    
    axes[1].fill_between(trades['exit_time'], 0, drawdown_pct, alpha=0.5, color='red')
    axes[1].plot(trades['exit_time'], drawdown_pct, linewidth=1.5, color='darkred')
    axes[1].set_title('回撤曲线', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('时间', fontsize=12)
    axes[1].set_ylabel('回撤 (%)', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    axes[1].invert_yaxis()
    
    plt.tight_layout()
    plt.show()
    
    print(f'最大回撤: {drawdown_pct.min():.2f}%')

## 4. 收益分布分析

In [ ]:
if trades_path.exists():
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 盈亏分布直方图
    axes[0, 0].hist(trades[trades['pnl'] > 0]['pnl'], bins=30, alpha=0.7, color='green', label='盈利')
    axes[0, 0].hist(trades[trades['pnl'] <= 0]['pnl'], bins=30, alpha=0.7, color='red', label='亏损')
    axes[0, 0].set_title('盈亏分布', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('P&L (¥)')
    axes[0, 0].set_ylabel('频数')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 盈亏百分比分布
    axes[0, 1].hist(trades[trades['pnl_pct'] > 0]['pnl_pct'], bins=30, alpha=0.7, color='green', label='盈利')
    axes[0, 1].hist(trades[trades['pnl_pct'] <= 0]['pnl_pct'], bins=30, alpha=0.7, color='red', label='亏损')
    axes[0, 1].set_title('盈亏百分比分布', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('P&L %')
    axes[0, 1].set_ylabel('频数')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 持仓时间分布
    axes[1, 0].hist(trades['bars_held'], bins=30, alpha=0.7, color='steelblue')
    axes[1, 0].set_title('持仓时间分布', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('持仓K线数')
    axes[1, 0].set_ylabel('频数')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 退出原因统计
    exit_counts = trades['exit_reason'].value_counts()
    axes[1, 1].pie(exit_counts, labels=exit_counts.index, autopct='%1.1f%%', startangle=90)
    axes[1, 1].set_title('退出原因分布', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

## 5. 时间分析

In [ ]:
if trades_path.exists():
    # 按小时统计
    trades['hour'] = trades['entry_time'].dt.hour
    hourly_pnl = trades.groupby('hour')['pnl'].agg(['sum', 'count', 'mean'])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 每小时盈亏
    axes[0].bar(hourly_pnl.index, hourly_pnl['sum'], alpha=0.7, color='steelblue')
    axes[0].set_title('每小时总盈亏', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('小时')
    axes[0].set_ylabel('总盈亏 (¥)')
    axes[0].grid(True, alpha=0.3)
    
    # 每小时交易数
    axes[1].bar(hourly_pnl.index, hourly_pnl['count'], alpha=0.7, color='coral')
    axes[1].set_title('每小时交易数', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('小时')
    axes[1].set_ylabel('交易数')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 6. 多空对比

In [ ]:
if trades_path.exists():
    # 多空统计
    long_trades = trades[trades['position'] == 'Long']
    short_trades = trades[trades['position'] == 'Short']
    
    print('多空对比分析')
    print('='*70)
    print(f'\n做多:')
    print(f'  交易数: {len(long_trades):,}')
    print(f'  总盈亏: ¥{long_trades["pnl"].sum():,.0f}')
    print(f'  平均盈亏: ¥{long_trades["pnl"].mean():,.0f}')
    print(f'  胜率: {(long_trades["pnl"] > 0).sum() / len(long_trades) * 100:.2f}%' if len(long_trades) > 0 else 'N/A')
    
    print(f'\n做空:')
    print(f'  交易数: {len(short_trades):,}')
    print(f'  总盈亏: ¥{short_trades["pnl"].sum():,.0f}')
    print(f'  平均盈亏: ¥{short_trades["pnl"].mean():,.0f}')
    print(f'  胜率: {(short_trades["pnl"] > 0).sum() / len(short_trades) * 100:.2f}%' if len(short_trades) > 0 else 'N/A')
    print('='*70)

## 💡 总结

本笔记本展示了：

1. ✅ 基础统计分析（胜率、盈亏比、盈利因子）
2. ✅ 权益曲线和回撤分析
3. ✅ 收益分布可视化
4. ✅ 时间维度分析
5. ✅ 多空对比

### 下一步优化方向

- 🎯 调整止盈止损参数
- 📊 优化信号生成阈值
- ⚖️ 改进仓位管理
- 🔄 测试不同时间周期

---

**完整教程系列完成！Happy Trading! 🎉**